## Collaborative Filtering

In [1]:
# import pandas
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np

We will create an recommender engine based on Item Based Collaborative Filtering (IBCF) which searches for the most similar books based on the user ratings. We can download the data from [here](https://drive.google.com/file/d/1WvTmAfO09TCX7xp7uu06__ziic7JnrL5/view?usp=sharing).

In [2]:
book_ratings = pd.read_csv('BX-CSV-Dump/BX-Book-Ratings.csv',sep=";", encoding="latin")
books = pd.read_csv('BX-CSV-Dump/BX-Books.csv',sep=";", encoding="latin", error_bad_lines=False)

/var/folders/fm/m7sbfrtn37dcz2lvxhvj6q0m0000gn/T/ipykernel_49209/3806774795.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv('BX-CSV-Dump/BX-Books.csv',sep=";", encoding="latin", error_bad_lines=False)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 1

In [3]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
book_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


* Explore both datasets

In [5]:
book_ratings.shape

(1149780, 3)

In [6]:
books.shape

(271360, 8)

In [7]:
books_merge = pd.merge(book_ratings, books, on='ISBN')

In [8]:
books_merge.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [9]:
books_merge = books_merge[['User-ID','ISBN','Book-Rating', 'Book-Title']]

In [10]:
books_merge.head()

,User-ID,ISBN,Book-Rating,Book-Title
0,276725,034545104X,0,Flesh Tones: A Novel
1,2313,034545104X,5,Flesh Tones: A Novel
2,6543,034545104X,0,Flesh Tones: A Novel
3,8680,034545104X,5,Flesh Tones: A Novel
4,10314,034545104X,9,Flesh Tones: A Novel


In [73]:
books_merge.isna().sum()

User-ID        0
ISBN           0
Book-Rating    0
Book-Title     0
dtype: int64

In [135]:
book_ratingCount = books_merge.groupby('ISBN')['Book-Rating'].count().reset_index(name='totalRatingCount')

In [136]:
book_ratingCount.sort_values(by='totalRatingCount', ascending=False).head(10)

,ISBN,totalRatingCount
215952,0971880107,2502
38570,0316666343,1295
70798,0385504209,883
7344,0060928336,732
32370,0312195516,723
87397,044023722X,647
21342,0142001740,615
145042,067976402X,614
133142,0671027360,586
93847,0446672211,585


In [138]:
books_with_ratingCount = books_merge.merge(book_ratingCount, left_on='ISBN', right_on='ISBN', how='inner')

In [139]:
books_with_ratingCount.head()

,User-ID,ISBN,Book-Rating,Book-Title,totalRatingCount
0,276725,034545104X,0,Flesh Tones: A Novel,60
1,2313,034545104X,5,Flesh Tones: A Novel,60
2,6543,034545104X,0,Flesh Tones: A Novel,60
3,8680,034545104X,5,Flesh Tones: A Novel,60
4,10314,034545104X,9,Flesh Tones: A Novel,60


In [140]:
books_with_ratingCount['totalRatingCount'].count()

1031136

In [141]:
pop_threshold = 50
popBooks = books_with_ratingCount[books_with_ratingCount['totalRatingCount']>pop_threshold]
popBooks.head()

,User-ID,ISBN,Book-Rating,Book-Title,totalRatingCount
0,276725,034545104X,0,Flesh Tones: A Novel,60
1,2313,034545104X,5,Flesh Tones: A Novel,60
2,6543,034545104X,0,Flesh Tones: A Novel,60
3,8680,034545104X,5,Flesh Tones: A Novel,60
4,10314,034545104X,9,Flesh Tones: A Novel,60


In [142]:
popBooks.shape

(231876, 5)

In [11]:
# ratings_mean_count = pd.DataFrame(books_merge.groupby('ISBN')['Book-Rating'].mean())
# ratings_mean_count['rating_count'] = pd.DataFrame(books_merge.groupby('ISBN')['Book-Rating'].count())

* create dataframe with name 'df_book_features' from book_ratings that have `ISBN` as index, `User-ID` as columns and values are `Book-Rating`.
    - The data are quite big so it's OK to use a sample only in case your PC has limited RAM.


In [183]:
books_merge_sample = popBooks.sample(frac=0.8, random_state=15)

In [184]:
popBooks.shape

(231876, 5)

In [185]:
books_merge_sample.shape

(185501, 5)

In [186]:
books_merge_sample.head()

,User-ID,ISBN,Book-Rating,Book-Title,totalRatingCount
230672,73127,0449912558,0,The Sparrow,106
130339,106424,0684800713,7,Sun Also Rises,62
81525,66912,0440211727,0,A Time to Kill,517
47258,268691,0316096199,8,Lucky : A Memoir,301
124840,113270,0385498802,0,Bee Season: A Novel,175


In [238]:
books_merge_sample['ISBN'] = books_merge_sample['ISBN'].astype(str)
books_merge_sample['Book-Title'] = books_merge_sample['Book-Title'].astype(str)

In [239]:
df_book_features = books_merge_sample.pivot_table(index='ISBN', columns='User-ID', values='Book-Rating').fillna(0)

In [240]:
df_book_features.shape

(2101, 37468)

In [241]:
df_book_features.head()

User-ID,9,16,17,26,32,39,42,44,51,67,...,278813,278819,278828,278832,278836,278843,278844,278846,278851,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
000649840X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002026478X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0020442203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0028604199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


* create the instance of the NearestNeighbors class

In [242]:
NN = NearestNeighbors(n_neighbors=6, metric='cosine')

* fit the NearestNeighbors using'df_book_features'

In [243]:
df_book_features.head()

User-ID,9,16,17,26,32,39,42,44,51,67,...,278813,278819,278828,278832,278836,278843,278844,278846,278851,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
000649840X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002026478X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0020442203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0028604199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [244]:
NN.fit(df_book_features)

NearestNeighbors(metric='cosine', n_neighbors=6)

* create function that returns top 5 most similar movies (according to KNN model) for selected ISBN
    * the input will be Book-Title from the DataFrame books 
    * the output will be the Book-Titles of the top 5 most similar books.
    * for every book in the top 5 most similar books, print also the distance from the selected book (ISBN we chose as input to the function)

In [274]:
books_merge_sample[books_merge_sample['ISBN']=='0439139597'].head()

,User-ID,ISBN,Book-Rating,Book-Title,totalRatingCount
194814,146230,0439139597,0,Harry Potter and the Goblet of Fire (Book 4),194
194750,43806,0439139597,0,Harry Potter and the Goblet of Fire (Book 4),194
194820,158295,0439139597,0,Harry Potter and the Goblet of Fire (Book 4),194
194866,215986,0439139597,0,Harry Potter and the Goblet of Fire (Book 4),194
194800,116904,0439139597,10,Harry Potter and the Goblet of Fire (Book 4),194


In [246]:
# create book title and isbn dictionary
isbn_title = pd.Series(books_merge_sample['ISBN'].values, index=books_merge_sample['Book-Title']).to_dict()

In [250]:
import random
random.sample(isbn_title.items(), k=2)

[('Life,the Universe and Everything', '0330267388'),
 ('The Stand (The Complete and Uncut Edition)', '0385199570')]

In [234]:
isbn_title['Harry Potter and the Goblet of Fire (Book 4)']

'0439139597'

In [194]:
isbn_title["Harry Potter and the Sorcerer's Stone (Book 1)"]

'043936213X'

In [195]:
# testing isbn from title
isbn_title['Forbidden Fruit']

'1551660717'

In [231]:
# testing title from ISBN
print([k for k,v in isbn_title.items() if v =='043936213X'])

[]


In [197]:
# locating with ISBN
df_book_features.loc[df_book_features.index=='043936213X']

User-ID,9,16,17,26,32,39,42,44,51,67,...,278813,278819,278828,278832,278836,278843,278844,278846,278851,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
043936213X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [223]:
# turn isbn to loc to an array across rows 
temp_query_arr = np.array(df_book_features.loc[df_book_features.index=='0439136369'].values).reshape(1,-1)
temp_query_arr

array([[0., 0., 0., ..., 0., 0., 0.]])

In [224]:
temp_query_arr.shape

(1, 37468)

In [225]:
# finding distances and indices on a query
distances, indices = NN.kneighbors(temp_query_arr)

In [226]:
indices

array([[ 856,  854,  858, 1617,  859,  857]])

In [227]:
indices.flatten()

array([ 856,  854,  858, 1617,  859,  857])

In [228]:
# this gets the ISBN from index
df_book_features.iloc[860].name

'043936213X'

In [229]:
# getting list of ISBN from index of KNN
rec_isbn_list = [df_book_features.iloc[x].name for x in indices.flatten()]
rec_isbn_list

['0439136369',
 '0439064872',
 '0439139600',
 '059035342X',
 '043935806X',
 '0439139597']

In [230]:
# print titles based on  ISBN dictionary
for x in range(0, len(rec_isbn_list)):
    print([title for title, isbn in isbn_title.items() if isbn==rec_isbn_list[x]])

[]
['Harry Potter and the Chamber of Secrets (Book 2)']
[]
["Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))"]
['Harry Potter and the Order of the Phoenix (Book 5)']
['Harry Potter and the Goblet of Fire (Book 4)']


In [209]:
list(distances.flatten())

[0.0,
 0.8944902112520905,
 0.913662659552894,
 0.9143920955024528,
 0.9157364971169429,
 0.9180379562975687]

In [285]:
df_book_features.head()

User-ID,9,16,17,26,32,39,42,44,51,67,...,278813,278819,278828,278832,278836,278843,278844,278846,278851,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
000649840X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002026478X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0020442203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0028604199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [289]:
df_book_features.index=='000649840X'

array([ True, False, False, ..., False, False, False])

In [320]:
ISBN_title_lookup = books_merge_sample.groupby('ISBN')['Book-Title'].unique().reset_index()

In [321]:
ISBN_title_lookup.head()

,ISBN,Book-Title
0,000649840X,[Angelas Ashes]
1,002026478X,[AGE OF INNOCENCE (MOVIE TIE-IN)]
2,0020442203,"[Lion, the Witch and the Wardrobe]"
3,002542730X,[Politically Correct Bedtime Stories: Modern T...
4,0028604199,[Once upon a More Enlightened Time: More Polit...


In [370]:
ISBN_title_lookup.loc[ISBN_title_lookup['ISBN']=='043935806X', 'Book-Title'].item()

array(['Harry Potter and the Order of the Phoenix (Book 5)'], dtype=object)

In [354]:
df_book_features.head()

User-ID,9,16,17,26,32,39,42,44,51,67,...,278813,278819,278828,278832,278836,278843,278844,278846,278851,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
000649840X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002026478X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0020442203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0028604199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [355]:
books_merge_sample.head()

,User-ID,ISBN,Book-Rating,Book-Title,totalRatingCount
230672,73127,0449912558,0,The Sparrow,106
130339,106424,0684800713,7,Sun Also Rises,62
81525,66912,0440211727,0,A Time to Kill,517
47258,268691,0316096199,8,Lucky : A Memoir,301
124840,113270,0385498802,0,Bee Season: A Novel,175


In [364]:
indices.flatten()[0]

856

In [366]:
df_book_features.iloc[1230].name

'0451161351'

In [403]:
def recommender(title):
    # turn title to isbn and get the array
    isbn = isbn_title[title]
    isbn_arr = np.array(df_book_features.loc[df_book_features.index==isbn]).reshape(1,-1)
    
    # get distances and indices for array from neighbours
    dist, ind = NN.kneighbors(isbn_arr, n_neighbors=6)
    
    # get ISBN from indices
    rec_isbn = [(df_book_features.iloc[x].name) for x in ind.flatten()]

    # get titles from ISBN look-up df
    rec_titles = [ISBN_title_lookup.loc[ISBN_title_lookup['ISBN']==x, 'Book-Title'].item()[0] for x in rec_isbn]
    
    
    for x in range(0, len(rec_isbn)):
        if x == 0:
            print(f"Recommendations for: {title}")
        else: 
            print("{0}. {1} (ISBN:{2}) with distance {3}".format(
                x, rec_titles[x], rec_isbn[x], list(dist.flatten())[x]
            ))
      
    return



* Apply the function to book of your choice

In [404]:
recommender("Fight Club") # seems good, recommended other Palahniuk books # prints could be better 

Recommendations for: Fight Club
1. Invisible Monsters (ISBN:0393319296) with distance 0.723143183600372
2. Survivor : A Novel (ISBN:0385498721) with distance 0.7914836874042523
3. Honor Among Thieves (ISBN:0061092045) with distance 0.8821086652763784
4. Night Chills (ISBN:0425098648) with distance 0.8823689699441127
5. Vittorio the Vampire: New Tales of the Vampires (ISBN:0375401601) with distance 0.8848411145166594
